In [1]:
import pandas as pd


df_all = pd.read_csv(".data/available_TA_all.csv")
df_all = df_all[df_all["relevant"]==2]
df_topic = pd.read_csv("df_introduction.csv")
df_all.head()

,T_id,doi,title,abstract,authors,year,publisher,relevant,wos,scopus,pubmed,introduction,methods,results,discuession,conclustion
0,TA000000,10.3390/foods10061207,Yeast Particle Encapsulation of Scaffolded Ter...,Terpenes are naturally occurring compounds pro...,"Soto, Ernesto R, Rus, Florentina, Li, Hanchen,...",2021,"Foods (Basel, Switzerland)",2,True,True,True,"[""**1. introduction** Terpenes are a large cla...","['**2. materials and methods** ', '*2.1. mater...",To investigate the toxicity of M. alternifolia...,['**4. discussion** Terpenes are natural produ...,['**5. conclusions** Yeast particles can be us...
1,TA000001,10.3390/molecules23112838,Volatile Terpenes and Terpenoids from Workers ...,Twenty-one volatile terpenes and terpenoids we...,"Zhao, Rui, Lu, Lihua, Shi, Qingxing, Chen, Jia...",2018,"Molecules (Basel, Switzerland)",2,True,True,True,['**1. introduction** Terpenes and terpenoids ...,"['**4. materials and methods** ', '*4.1. ants*...",To investigate the toxicity of M. alternifolia...,"[""**3. discussion** Twenty-one terpenes and te...","[""**5. conclusions** In summary, twenty-one vo..."
2,TA000002,10.3390/molecules27113580,Yeast Particles Hyper-Loaded with Terpenes for...,Yeast particles (YPs) are 3−5 µm hollow and po...,"Soto, Ernesto R, Rus, Florentina, Ostroff, Gary R",2022,"Molecules (Basel, Switzerland)",2,True,True,True,"[""**1. introduction** Terpenes are a large cla...",['mixture (1.65 g get at a composition of 2:1:...,To investigate the toxicity of M. alternifolia...,['**2. results and discussion** '],['**4. conclusions** Yeast particles can be us...
3,TA000003,10.3390/molecules23112803,Olfactory Impact of Terpene Alcohol on Terpene...,The key point of our work was evaluating the i...,"Niu, Yunwei, Sun, Xiaoxin, Xiao, Zuobing, Wang...",2018,"Molecules (Basel, Switzerland)",2,True,True,True,['**1. introduction** Chrysanthemums are of th...,"['**2. materials and methods** ', '*2.1. chemi...",To investigate the toxicity of M. alternifolia...,['**3. results and discussion** used in plsr m...,['**4. conclusions** The addition of isoborneo...
4,TA000004,10.1007/s10532-013-9619-4,Using volatile organic compounds to enhance at...,"The effect of the terpenes alpha-pinene, eucal...","Tortella, G. R.; Rubilar, O.; Stenstrom, J.; C...",2013,Biodegradation,2,True,True,False,['introduction A factor influencing the succes...,"['materials and methods ', 'chemicals Analytic...",To investigate the toxicity of M. alternifolia...,NaN,"['<span id=""page-8-0""></span>conclusion Adding..."


In [2]:
len(df_all[~df_all["discuession"].isna()])

23190

In [3]:
df_discuession = df_all[~df_all["discuession"].isna()]

In [4]:
rows = df_discuession.sample(2)
rows

,T_id,doi,title,abstract,authors,year,publisher,relevant,wos,scopus,pubmed,introduction,methods,results,discuession,conclustion
34168,TA050383,10.3390/ijms19061574,The Extracts and Major Compounds Derived from ...,"Astragali Radix (AR) is a widely used ""Qi-invi...","Huang, Yun, Kwan, Kenneth Kin Leung, Leung, Ka...",2018,International journal of molecular sciences,2,False,False,True,"Astragali Radix (AR), derived from the dry roo...",High performance liquid chromatography (HPLC)-...,"The contents of calycosin, astragaloside IV, g...",AR is well known for its “Qi-invigorating” act...,NaN
1214,TA001417,10.1002/anie.202102453,Discovery and Biosynthesis of a Structurally D...,"A new bicyclic diterpenoid, benditerpenoic aci...","Zhu, Chenxi; Xu, Baofu; Adpressa, Donovon A.; ...",2021,Angewandte Chemie - International Edition,2,True,True,False,"['introduction Terpenoids, found in all branch...",['isolation and structural elucidation of bend...,To investigate the toxicity of M. alternifolia...,"['results and discussion ', 'structurally rela...","['conclusion In summary, we discovered a bacte..."


In [5]:
prompt = """
The topic is {}
=================
Discuession:{}
Based on the above information:
 - First, Divide the text reasonably, each paragraph should include the brief summary and the original text. 
 - Do not omit any information in original text, but you can clean it.
 - It is recommended to merge related paragraphs. Each output paragraph should be no less than 500 words.
 - No output in any format other than JSON is required. Translate it into English as a prompt.
EXAMPLE JSON OUTPUT:
{{
    "0001": {{
        "summary":"",
        "original_text":""，
    }},
    "0002": {{
        "summary":"",
        "original_text":"",
    }},
}}
The response is:
"""
llm_data = []
for _, row in df_discuession.iterrows():
    T_id = row["T_id"]
    topic = df_topic[df_topic["T_id"]==T_id]["topic"].values[0]
    content = row["discuession"]  ### 重要，记得修改
    llm_input = prompt.format(topic, content)
    llm_data.append({
        "custom_id": T_id,
        "llm_input": llm_input
    })
len(llm_data)



23190

In [6]:
llm_data[13]["llm_input"]

'\nThe topic is Selected cannabis terpenes synergize with THC to enhance CB1 receptor activation.\n=================\nDiscuession:[\'**4. discussion** The current study evaluated the role of terpenes in CB1 receptor -mediated functions, studying terpenes as direct agonists of CB1 receptors and as modulators of THC agonism. The results are discussed in view of methodological considerations and in comparison to prior reports, elaborating on plausible mechanisms of action and important therapeutic implications. Selection of the compositions to be tested. In order to optimally evaluate the role of terpenes on CB1 receptor and their modulation of THC-derived CB1 receptor activation, it is important to properly select (1) a representative set of diverse cannabis terpene; (2) a terpene to cannabinoid ratio representative of the natural cannabis plants, being in the order of 1/10 weight/ weight; and (3) concentrations within the solubility limits of the various reagents. The selections made he

In [7]:
def data_to_jsonl_volcengine(datas:list, save_file_path=".tmp.jsonl"):
    import json
    # 火山引擎要求的jsonl格式
    with open(save_file_path, "w", encoding="utf-8") as f:
        for _data in datas:
            custom_id = _data["custom_id"]
            user_input = _data["llm_input"]
            _input = {
                "custom_id": custom_id,
                "body": {
                    "messages": [
                        {"role": "system","content": ""},
                        {"role": "user","content": user_input}
                    ],
                    "max_tokens": 10000,
                    "temperature": 0
                },
            }
            f.write(json.dumps(_input, ensure_ascii=False) + "\n")
    return

data_to_jsonl_volcengine(llm_data, "h_discuession_input.jsonl")

In [1]:
import json
from tqdm import tqdm

discuession_datas = []
err_ids = []

with open("h_discuession_output.jsonl", "r") as r:
    for line in tqdm(r.readlines()):
        llm_data = json.loads(line)
        T_id = llm_data["custom_id"]
        llm_msg = llm_data["response"]["body"]["choices"][0]["message"]["content"]
        llm_msg = llm_msg.replace("`","").replace("json","").replace("\n","")
        try:
            msg_data = eval(llm_msg)
            _data = {"T_id": T_id}
            for k, v in msg_data.items():
                _data[k] = v
            discuession_datas.append(_data)
        except:
            err_ids.append(T_id)

len(err_ids)

100%|██████████| 23190/23190 [00:02<00:00, 7943.53it/s]


201

In [2]:
import pandas as pd

discuession_df = pd.DataFrame(discuession_datas)

discuession_split_data = []
for _, row in tqdm(discuession_df[:].iterrows()):
    T_id = row["T_id"]
    for column_name, value in row.items():  # 直接遍历Series的键值对
        if column_name == "T_id":
            continue
        if pd.isna(row[column_name]):
            break
        else:
            _data = {}
            _data["discuession_id"] = T_id + "_D_" + column_name
            _data["summary"] = value["summary"]
            _data["original_text"] = value["original_text"] if "original_text" in value else None
            discuession_split_data.append(_data)

# method_df
discuession_split_df = pd.DataFrame(discuession_split_data)
discuession_split_df




22989it [00:00, 33691.96it/s]


,discuession_id,summary,original_text
0,TA002790_D_0001,The discussion highlights the significant impa...,**Discussion of results** Analysis of the obta...
1,TA002712_D_0001,The discussion focuses on the development of t...,The topic is Development of terpene- and terpe...
2,TA002712_D_0002,The results and discussion section delves into...,**results and discussion**
3,TA003126_D_0001,The discussion highlights the critical role of...,"**journal name article** critical, but when it..."
4,TA003233_D_0001,The discussion section introduces the comparat...,**discussion** The comparative analysis of ter...
...,...,...,...
53679,TA032176_D_0005,This section covers the X-ray diffraction (XRD...,3.5. x-ray diffraction All samples were analyz...
53680,TA032176_D_0006,This section discusses the differential scanni...,3.6. dsc Differential scanning calorimetry (DS...
53681,TA050531_D_0001,The discussion begins by outlining the metabol...,Plants subjected to saline conditions undergo ...
53682,TA050531_D_0002,The text delves into salinity adaptation strat...,Salinity adaptation can be obtained through Na...


In [7]:
discuession_split_df.to_csv(".data/chunks_discuession.csv", index=False)